In [ ]:
# imports

import tensorflow as tf
import tensorflow_models as tfm
import numpy as np



In [4]:
# download the dataset

!wget http://images.cocodataset.org/zips/train2014.zip
!wget http://images.cocodataset.org/zips/val2014.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip
!wget http://images.cocodataset.org/zips/test2014.zip
!wget http://images.cocodataset.org/annotations/image_info_test2014.zip


/home/samer/anaconda3/envs/tensorflow/lib/python3.9/pty.py:85: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


--2024-09-03 01:43:37--  http://images.cocodataset.org/zips/train2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 16.182.39.33, 54.231.225.209, 3.5.28.204, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|16.182.39.33|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13510573713 (13G) [application/zip]
Saving to: ‘train2014.zip’

train2014.zip       100%[===================>]  12.58G  4.28MB/s    in 59m 42s 

2024-09-03 02:43:19 (3.60 MB/s) - ‘train2014.zip’ saved [13510573713/13510573713]

--2024-09-03 02:43:19--  http://images.cocodataset.org/zips/val2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.166.153, 3.5.28.168, 3.5.25.173, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.166.153|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6645013297 (6.2G) [application/zip]
Saving to: ‘val2014.zip’

val2014.zip          22%[===>                ]   1.40G

In [ ]:
tfm.vision.decoders.FPN(
    input_specs: Mapping[str, tf.TensorShape],
    min_level: int = 3,
    max_level: int = 7,
    num_filters: int = 256,
    fusion_type: str = 'sum',
    use_separable_conv: bool = False,
    use_keras_layer: bool = False,
    activation: str = 'relu',
    use_sync_bn: bool = False,
    norm_momentum: float = 0.99,
    norm_epsilon: float = 0.001,
    kernel_initializer: str = 'VarianceScaling',
    kernel_regularizer: Optional[tf.keras.regularizers.Regularizer] = None,
    bias_regularizer: Optional[tf.keras.regularizers.Regularizer] = None,
    **kwargs
)